<a href="https://colab.research.google.com/github/rllima/IF704/blob/main/Classification%20step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [13]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from google.colab import drive

In [4]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [16]:
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [20]:
#Download do dataset
import requests, zipfile, io
r = requests.get("http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("/content/drive/My Drive/IF704")
z.close()

In [21]:
!ls "/content/drive/My Drive/IF704/"

testdata.manual.2009.06.14.csv	training.1600000.processed.noemoticon.csv


In [35]:
df = pd.read_csv("/content/drive/My Drive/IF704/training.1600000.processed.noemoticon.csv",
                 names = ["target","id","date","query","user","text"], header=None, encoding='latin-1')

In [36]:
df.head()

,target,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [33]:
###Preparando o dataset para tensorflow

In [38]:
target = df.pop('target')

In [40]:
dataset = tf.data.Dataset.from_tensor_slices((df.text.values, target.values))

In [41]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: b"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", Target: 0
Features: b"is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", Target: 0
Features: b'@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds', Target: 0
Features: b'my whole body feels itchy and like its on fire ', Target: 0
Features: b"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. ", Target: 0


In [46]:
DATASET_SIZE = df.shape[0]

In [50]:
train_size = int(0.8 * DATASET_SIZE)
test_size = int(0.2 * DATASET_SIZE)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
test_dataset = test_dataset.take(test_size)

In [44]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [51]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [52]:
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [53]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [54]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'i', 'to', 'the', 'a', 'my', 'and', 'you', 'is', 'it',
       'in', 'for', 'of', 'im', 'on', 'me', 'so', 'have', 'that'],
      dtype='<U13')

In [55]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

NameError: ignored